# [06] Multi-Label Text Classification with Qwen-1.5-0.5B
**Objective:** To train and optimize a multi-label transformer model for simultaneously detecting advertisement, relevance, and rant violations in reviews, using a combination of text features and metadata.
**Input:**
- `../data/processed/all_reviews_with_labels.parquet` (from Notebook 04)
- `../data/processed/synthetic_reviews.json` (from Notebook 05)
**Output:** 
- Trained Qwen-1.5-0.5B model weights
- Optimal threshold values for each class
- Performance metrics and evaluation results

## Introduction
This notebook implements the core machine learning component of our content moderation system. We face a complex multi-label classification problem with severe class imbalance, requiring specialized handling of both text and metadata features.

The process involves several key stages:

1.  **Data Integration & Feature Engineering:** We combine our original labeled data with synthetically generated examples to balance the classes. We then engineer crucial meta-features from the raw text (URL counts, phone numbers, capitalization ratios, etc.) that provide strong signals for certain violation types.

2.  **Stratified Train-Test Split:** We employ multi-label stratification to ensure that all combinations of our three target labels (`is_ad`, `is_relevant`, `is_rant`) are proportionally represented in both training and testing sets, which is critical for reliable evaluation.

3.  **Model Architecture:** We implement a hybrid model that combines:
    - A **Qwen-1.5-0.5B transformer** for processing review text and extracting deep semantic features.
    - A **custom neural network** for processing the engineered meta-features.
    - A **fusion layer** that integrates both information streams for final prediction.

4.  **Threshold Optimization:** Due to extreme class imbalance, we perform precision-recall analysis to find optimal probability thresholds for each class (0.756 for ads, 0.1 for relevance, and 0.852 for rants) rather than using the default 0.5.

The final model provides an automated content moderation solution that balances detection sensitivity with precision constraints through learned business rules.

### Dependencies

In [1]:
!pip install -q transformers accelerate datasets peft torch tensorboard iterative-stratification scikit-learn plotly optuna
!pip install --upgrade --quiet nltk textblob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.4/247.4 kB 23.9 MB/s eta 0:00:00


In [93]:
# ===== Standard Library =====
import os
import re
import gc
import shutil
import psutil
import yaml
import json

# ===== Data Processing & Utilities =====
import numpy as np
import pandas as pd
import nltk
from textblob import TextBlob
from datasets import Dataset
from sklearn.preprocessing import StandardScaler, LabelEncoder

# ===== PyTorch & CUDA =====
import torch
import torch.nn as nn
from torch.utils.data import IterableDataset, DataLoader
from torch.cuda import amp
from torch.cuda.amp import autocast, GradScaler
from torch.optim import AdamW
from tqdm import tqdm

# ===== Transformers & NLP Models =====
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    pipeline,
    get_scheduler
)

# ===== Hugging Face PEFT (LoRA) =====
from peft import LoraConfig, get_peft_model

# ===== Evaluation Metrics =====
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    precision_recall_fscore_support,
    average_precision_score,
    precision_recall_curve
)

# ===== ML Utilities =====
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

# ===== Google Colab =====
from google.colab import files

In [3]:
uploaded = files.upload()

Saving all_reviews_with_labels_normalised.csv to all_reviews_with_labels_normalised.csv
Saving synthetic_combined.csv to synthetic_combined.csv


### 1. Load Data

In [4]:
all_reviews = list(uploaded.keys())[0]
synthetic_combined = list(uploaded.keys())[1]

In [5]:
full_df = pd.read_csv(all_reviews)
full_df = full_df.dropna(subset=['rating']).reset_index(drop=True)

print(f"Loaded {all_reviews} with {len(full_df)} rows")
print(full_df.isnull().sum())

Loaded all_reviews_with_labels_normalised.csv with 11667 rows
review_text             0
rating                  0
has_photo               0
author_name             0
user_review_count       0
business_name           0
category                0
source                  0
review_id               0
comprehensive_review    0
is_ad                   0
is_relevant             0
is_rant                 0
is_legit                0
dtype: int64


In [6]:
synthetic_df = pd.read_csv(synthetic_combined)

def s(col):
    return synthetic_df[col].fillna("NA").astype(str).str.strip()

has_photo_str = np.where(synthetic_df["has_photo"].fillna(False), "yes", "no")
MAX_REVIEW_CHARS = 2000
review_text_clean = s("review_text").str.replace(r"\s+", " ", regex=True).str[:MAX_REVIEW_CHARS]

synthetic_df["comprehensive_review"] = (
    "[Business] " + s("business_name") +
    " | [Category] " + s("category") +
    " | [Rating] " + s("rating") +
    " | [Author] " + s("author_name") +
    " | [User Review Count] " + s("user_review_count") +
    " | [Has Photo] " + pd.Series(has_photo_str, index=synthetic_df.index) +
    " | [Source] " + s("source") +
    " | [Review] " + review_text_clean
).str.replace(r"\s+\|\s+\[Review\]\s+NA$", "", regex=True)

print(f"Loaded {synthetic_combined} with {len(synthetic_df)} rows")
print(synthetic_df.isnull().sum())

Loaded synthetic_combined.csv with 714 rows
review_text             0
rating                  0
has_photo               0
author_name             0
user_review_count       0
business_name           0
category                0
source                  0
review_id               0
is_ad                   0
is_rant                 0
is_legit                0
is_relevant             0
comprehensive_review    0
dtype: int64


In [7]:
to_clean_df = full_df.dropna(subset=['review_text', 'is_ad', 'is_relevant', 'is_rant', 'is_legit'])

to_clean_df.head()

,review_text,rating,has_photo,author_name,user_review_count,business_name,category,source,review_id,comprehensive_review,is_ad,is_relevant,is_rant,is_legit
0,Love the convenience of this neighborhood carw...,4.0,False,Doug Schmidt,1.0,"Auto Spa Speedy Wash - Harvester, MO",['Car wash'],google,1001,"[Business] Auto Spa Speedy Wash - Harvester, M...",False,True,False,True
1,"2 bathrooms (for a large 2 story building), 1 ...",2.0,False,Duf Duftopia,1.0,Kmart,"['Discount store', 'Appliance store', 'Baby st...",google,1002,[Business] Kmart | [Category] ['Discount store...,True,True,True,False
2,My favorite pizza shop hands down!,5.0,False,Andrew Phillips,1.0,Papa’s Pizza,"['Pizza restaurant', 'Chicken wings restaurant...",google,1003,[Business] Papa’s Pizza | [Category] ['Pizza r...,False,True,False,True
3,BOTCHED INSTRUMENT REPAIR IS COSTING US HUNDRE...,1.0,False,Julie Heiland,1.0,The Music Place,['Musical instrument store'],google,1004,[Business] The Music Place | [Category] ['Musi...,False,True,True,False
4,Very unprofessional!!!!!,1.0,False,Alan Khasanov,1.0,Park Motor Cars Inc,['Used car dealer'],google,1005,[Business] Park Motor Cars Inc | [Category] ['...,False,True,True,False


### 2. Pre-Process Datafames

##### 2.1 Cleaning Functions

In [8]:
def normalize_whitespace(text):
    return re.sub(r'\s+', ' ', text).strip()

def clean_text(text):
    if pd.isna(text):
        return ""
    text = str(text)
    text = normalize_whitespace(text)
    return text

##### 2.2 Compute Basic Signals

In [9]:
def compute_basic_signals(text):
    url_count = len(re.findall(r'https?://\S+', text))
    phone_count = len(re.findall(r'\+?\d[\d\s-]{7,}\d', text))
    caps_ratio = sum(1 for c in text if c.isupper()) / max(len(text), 1)
    return url_count, phone_count, caps_ratio

##### 2.3 Sentiment Analysis

In [10]:
def add_textblob_sentiment(df, text_col="review_text", positive_threshold=0.9, negative_threshold=-0.9):
    def get_sentiment(text):
        if pd.isna(text) or not isinstance(text, str) or text.strip() == "":
            return 0.0, 0.0
        try:
            analysis = TextBlob(text)
            return analysis.sentiment.polarity, analysis.sentiment.subjectivity
        except Exception:
            return 0.0, 0.0

    sentiment_results = df[text_col].apply(get_sentiment)
    df["sentiment_polarity"], df["sentiment_subjectivity"] = zip(*sentiment_results)

    df["is_extreme_sentiment"] = df["sentiment_polarity"].apply(
        lambda x: 1 if x >= positive_threshold or x <= negative_threshold else 0
    )

    return df

##### Apply to Dataframe

In [11]:
def preprocess_reviews(df):
    df["clean_text"] = df["review_text"].apply(clean_text)
    signals = df["clean_text"].apply(compute_basic_signals)
    df["url_count"], df["phone_count"], df["caps_ratio"] = zip(*signals)
    return df

cleaned_df = preprocess_reviews(to_clean_df)
print(cleaned_df.head())

cleaned_synthetic_df = preprocess_reviews(synthetic_df)
print(cleaned_synthetic_df.head())

                                         review_text  rating  has_photo  \
0  Love the convenience of this neighborhood carw...     4.0      False   
1  2 bathrooms (for a large 2 story building), 1 ...     2.0      False   
2                 My favorite pizza shop hands down!     5.0      False   
3  BOTCHED INSTRUMENT REPAIR IS COSTING US HUNDRE...     1.0      False   
4                           Very unprofessional!!!!!     1.0      False   

       author_name  user_review_count                         business_name  \
0     Doug Schmidt                1.0  Auto Spa Speedy Wash - Harvester, MO   
1     Duf Duftopia                1.0                                 Kmart   
2  Andrew Phillips                1.0                          Papa’s Pizza   
3    Julie Heiland                1.0                       The Music Place   
4    Alan Khasanov                1.0                   Park Motor Cars Inc   

                                            category  source  review_id  \

In [12]:
# # Save as JSON
# output_json_path = os.path.join(labeled_input_folder, "cleaned_df.json")
# cleaned_df.to_json(output_json_path, orient="records", lines=True, force_ascii=False)
# print(f"JSON file saved to: {output_json_path}")

# # Save as Parquet
# output_parquet_path = os.path.join(labeled_input_folder, "cleaned_df.parquet")
# cleaned_df.to_parquet(output_parquet_path, index=False)
# print(f"Parquet file saved to: {output_parquet_path}")

### 3. Train-Test Split with Multi-Label Stratification

In [13]:
meta_cols = ["clean_text", "url_count","phone_count","caps_ratio","rating","has_photo","user_review_count"]
label_cols = ["is_ad", "is_relevant", "is_rant", "is_legit"]

X = cleaned_df.drop(columns=label_cols)
y = cleaned_df[label_cols].values

mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
train_val_idx, test_idx = next(mskf.split(X, y))

train_val_df = cleaned_df.iloc[train_val_idx].reset_index(drop=True)
test_df = cleaned_df.iloc[test_idx].reset_index(drop=True)

y_train_val = y[train_val_idx]
train_idx, val_idx = next(mskf.split(train_val_df.drop(columns=label_cols), y_train_val))

train_df_original = train_val_df.iloc[train_idx].reset_index(drop=True)
val_df = train_val_df.iloc[val_idx].reset_index(drop=True)

train_df = pd.concat([train_df_original, cleaned_synthetic_df], ignore_index=True).reset_index(drop=True)

print(f"Training set size: {train_df.shape} (includes synthetic data)")
print(f"Validation set size: {val_df.shape}")
print(f"Test set size: {test_df.shape}")

print(f"Synthetic data in validation set: {val_df['review_id'].isin(cleaned_synthetic_df['review_id']).any()}")
print(f"Synthetic data in test set: {test_df['review_id'].isin(cleaned_synthetic_df['review_id']).any()}")

Training set size: (8181, 18) (includes synthetic data)
Validation set size: (1867, 18)
Test set size: (2333, 18)
Synthetic data in validation set: False
Synthetic data in test set: False


### 4. Tokenisation

In [14]:
def simple_tokenize(text):
    text = str(text).lower()
    tokens = re.findall(r'\b[a-z]+\b', text)
    return tokens

train_df['tokens'] = train_df['clean_text'].apply(simple_tokenize)
test_df['tokens'] = test_df['clean_text'].apply(simple_tokenize)

In [15]:
print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

(8181, 19)
(1867, 18)
(2333, 19)


### Yuen Ning's model

In [60]:
class QwenWithMeta(nn.Module):
    def __init__(self, model_name, num_labels, meta_embedding_size):
        super().__init__()
        self.text_model = AutoModel.from_pretrained(model_name)
        hidden_size = self.text_model.config.hidden_size

        # Projection layer to ensure text and meta embeddings have the same size
        # (if they don't already)
        self.meta_projection = nn.Linear(meta_embedding_size, hidden_size)

        self.classifier = nn.Linear(hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, meta_embedding, **kwargs):
        text_outputs = self.text_model(input_ids=input_ids, attention_mask=attention_mask)
        text_emb = text_outputs.last_hidden_state[:, 0, :] # [CLS] token

        # Project the pre-computed meta embedding to the same space
        meta_emb = self.meta_projection(meta_embedding)

        combined_emb = text_emb + meta_emb
        logits = self.classifier(combined_emb)
        return {"logits": logits}

In [42]:
class MetaMLP(nn.Module):
    def __init__(self, meta_dim, hidden_size=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(meta_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size), # Output a fixed-size embedding
        )

    def forward(self, x):
        return self.net(x)

# 1. Initialize the meta embedder and pre-compute embeddings
meta_embedder = MetaMLP(meta_dim=len(meta_cols))
meta_embedder.eval() # Set to eval mode

def precompute_meta_embeddings(df, meta_embedder, meta_cols):
    """Pre-compute meta embeddings for a DataFrame"""
    with torch.no_grad(): # No need for gradients
        meta_tensor = torch.tensor(df[meta_cols].values.astype(np.float32))
        meta_embeddings = meta_embedder(meta_tensor)
    return meta_embeddings.numpy() # Return as numpy array for the dataset

# Pre-compute for train and validation
train_meta_emb = precompute_meta_embeddings(train_df_filtered, meta_embedder, meta_cols)
val_meta_emb = precompute_meta_embeddings(val_df_filtered, meta_embedder, meta_cols)

In [68]:
label_cols = ["is_ad", "is_relevant", "is_rant"]
meta_cols = ["url_count", "phone_count", "caps_ratio", "rating", "has_photo", "user_review_count"]

labels_array = np.array(train_df[label_cols])
pos_counts = labels_array.sum(axis=0)
neg_counts = len(labels_array) - pos_counts
pos_weight = torch.tensor(neg_counts / (pos_counts + 1e-8), dtype=torch.float)

train_df_filtered = train_df.drop(columns=["is_legit"])
val_df_filtered = val_df.drop(columns=["is_legit"])
test_df_filtered = test_df.drop(columns=["is_legit"])

def prepare_dataset(df, meta_embeddings):
    df = df.reset_index(drop=True).copy()
    # Add the pre-computed meta embeddings to the dataframe
    df['meta_embedding'] = list(meta_embeddings) # Store as a list of arrays
    return Dataset.from_pandas(df[["clean_text"] + label_cols + ["meta_embedding"]])

train_dataset = prepare_dataset(train_df_filtered, train_meta_emb)
val_dataset = prepare_dataset(val_df_filtered, val_meta_emb)

def preprocess(batch):
    tokenized = tokenizer(batch["clean_text"], truncation=True, padding="max_length", max_length=512)
    # Meta embeddings are already computed, just convert them to tensor
    tokenized["meta_embedding"] = torch.tensor(np.array(batch["meta_embedding"]), dtype=torch.float)
    return tokenized

train_dataset = train_dataset.map(preprocess, batched=True)
val_dataset = val_dataset.map(preprocess, batched=True)
test_dataset = val_dataset.map(preprocess, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"] + label_cols + ["meta_embedding"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"] + label_cols + ["meta_embedding"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"] + label_cols + ["meta_embedding"])


Map:   0%|          | 0/8181 [00:00<?, ? examples/s]

Map:   0%|          | 0/1867 [00:00<?, ? examples/s]

Map:   0%|          | 0/1867 [00:00<?, ? examples/s]

In [40]:
model_name = "Qwen/Qwen1.5-0.5B"
model = QwenWithMeta(model_name, num_labels=len(label_cols), meta_embedding_size=64)
config = LoraConfig(r=8, lora_alpha=32, lora_dropout=0.1)
model.text_model = get_peft_model(model.text_model, config)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model.text_model.config.pad_token_id = tokenizer.pad_token_id

In [63]:
def data_collator(batch):
    input_ids = torch.stack([item["input_ids"] for item in batch])
    attention_mask = torch.stack([item["attention_mask"] for item in batch])
    meta_embedding = torch.stack([item["meta_embedding"] for item in batch]) # Now getting the embedding
    labels = torch.stack([torch.tensor([item[col] for col in label_cols], dtype=torch.float) for item in batch])
    return {"input_ids": input_ids, "attention_mask": attention_mask, "meta_embedding": meta_embedding, "labels": labels}

class MultiLabelTrainer(Trainer):
    def __init__(self, *args, pos_weight=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.pos_weight = pos_weight

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        meta_embedding = inputs.pop("meta_embedding") # Changed from 'meta'
        outputs = model(**inputs, meta_embedding=meta_embedding) # Changed here
        logits = outputs["logits"]
        loss_fct = torch.nn.BCEWithLogitsLoss(pos_weight=self.pos_weight.to(logits.device))
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = torch.sigmoid(torch.tensor(logits)).numpy()
    preds = (probs > 0.5).astype(int)

    precision = precision_score(labels, preds, average='macro', zero_division=0)
    recall = recall_score(labels, preds, average='macro', zero_division=0)
    f1 = f1_score(labels, preds, average='macro', zero_division=0)
    print(f"Precision: {precision}, Recall: {recall}, F1: {f1}")

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

batch_size = 8
training_args = TrainingArguments(
    output_dir="./qwen_meta_multilabel",
    num_train_epochs=3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    fp16=True,
    save_strategy="epoch",
    eval_steps=500,
    logging_steps=100,
    remove_unused_columns=False,
    report_to="none"
)

trainer = MultiLabelTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    pos_weight=pos_weight
)

/tmp/ipython-input-406015554.py:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `MultiLabelTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


In [44]:
trainer.train()

Step,Training Loss
100,1.103400
200,0.891400
300,0.839300
400,0.861200
500,0.749100
600,0.663800
700,0.843800
800,0.710000
900,0.690900
1000,0.718800


TrainOutput(global_step=1536, training_loss=0.7464103102684021, metrics={'train_runtime': 2114.8145, 'train_samples_per_second': 11.605, 'train_steps_per_second': 0.726, 'total_flos': 0.0, 'train_loss': 0.7464103102684021, 'epoch': 3.0})

In [47]:
import torch
import json
from pathlib import Path

# Create a directory for your model
model_dir = Path("./qwen_meta_multilabel_final")
model_dir.mkdir(exist_ok=True)

# 1. Save the complete model state
torch.save({
    'model_state_dict': model.state_dict(),
    'text_model_config': model.text_model.config,
    'meta_embedding_size': 64,
    'num_labels': len(label_cols)
}, model_dir / "model_checkpoint.pth")

# 2. Save the tokenizer
tokenizer.save_pretrained(model_dir)

# 3. Save the label information
with open(model_dir / "label_cols.json", "w") as f:
    json.dump(label_cols, f)

# 4. Save the model class definition code (important for future loading)
with open(model_dir / "model_definition.py", "w") as f:
    f.write('''
import torch
import torch.nn as nn
from transformers import AutoModel
from peft import PeftModel

class QwenWithMeta(nn.Module):
    def __init__(self, model_name, num_labels, meta_embedding_size):
        super().__init__()
        self.text_model = AutoModel.from_pretrained(model_name)
        hidden_size = self.text_model.config.hidden_size
        self.meta_projection = nn.Linear(meta_embedding_size, hidden_size)
        self.classifier = nn.Linear(hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, meta_embedding):
        text_outputs = self.text_model(input_ids=input_ids, attention_mask=attention_mask)
        text_emb = text_outputs.last_hidden_state[:, 0, :] # [CLS] token
        meta_emb = self.meta_projection(meta_embedding)
        combined_emb = text_emb + meta_emb
        logits = self.classifier(combined_emb)
        return {"logits": logits}
''')

print("Model saved successfully!")

Model saved successfully!


In [48]:
shutil.make_archive('qwen_meta_multilabel_final', 'zip', './qwen_meta_multilabel_final')

files.download('qwen_meta_multilabel_final.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [105]:
checkpoint = torch.load("./qwen_meta_multilabel_final/model_checkpoint.pth", weights_only=False)
model_state_dict = checkpoint['model_state_dict']

model = QwenWithMeta(
    model_name=checkpoint['text_model_config']._name_or_path,
    num_labels=checkpoint['num_labels'],
    meta_embedding_size=checkpoint['meta_embedding_size']
)

model.load_state_dict(model_state_dict, strict=False)

trainer.model = model

In [64]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_model_preparation_time': 0.0041, 'eval_runtime': 57.393, 'eval_samples_per_second': 32.53, 'eval_steps_per_second': 4.077}


In [73]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for i in range(len(val_dataset)):
        item = val_dataset[i]

        # Prepare inputs
        inputs = {
            "input_ids": item["input_ids"].unsqueeze(0).to(device),
            "attention_mask": item["attention_mask"].unsqueeze(0).to(device),
            "meta_embedding": item["meta_embedding"].unsqueeze(0).to(device)
        }

        # Get prediction
        outputs = model(**inputs)
        logits = outputs["logits"]
        probs = torch.sigmoid(logits)
        preds = (probs > 0.5).int().cpu().numpy()
        all_preds.append(preds[0])

        # Get labels
        labels = [item["is_ad"], item["is_relevant"], item["is_rant"]]
        all_labels.append(labels)

# Convert to arrays
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Compute metrics
precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

print(f"Manual Evaluation Results:")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1: {f1:.4f}")

Manual Evaluation Results:
Precision: 0.3628
Recall: 0.7638
F1: 0.3332


### 5. Threshold Tuning

In [83]:
def safe_get_predictions_proba(model, tokenizer, texts, meta_embeddings, device='cuda', batch_size=16):
    """Safe prediction function with proper device and type handling"""
    model.eval()
    model.to(device)
    all_probs = []

    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size), desc="Getting predictions"):
            batch_texts = texts[i:i+batch_size]
            batch_meta_embeddings = meta_embeddings[i:i+batch_size]

            # Tokenize
            inputs = tokenizer(
                batch_texts,
                truncation=True,
                padding=True,
                max_length=512,
                return_tensors='pt'
            )

            # Convert meta embeddings to tensor
            meta_tensor = torch.tensor(batch_meta_embeddings, dtype=torch.float32)

            # Explicitly move to device
            inputs = {key: value.to(device) for key, value in inputs.items()}
            meta_tensor = meta_tensor.to(device)

            outputs = model(**inputs, meta_embedding=meta_tensor)
            probs = torch.sigmoid(outputs["logits"])

            # Convert to float32 before moving to CPU for numpy compatibility
            probs_float32 = probs.float().cpu().numpy()
            all_probs.extend(probs_float32)

    return np.array(all_probs)

In [90]:
class MultilabelThresholdTuner:
    def __init__(self, model, tokenizer, label_cols, meta_embedder, meta_cols, device='cuda'):
        self.model = model
        self.tokenizer = tokenizer
        self.label_cols = label_cols
        self.meta_embedder = meta_embedder
        self.meta_cols = meta_cols
        self.device = device
        self.best_thresholds = None
        self.best_metrics = None

    def get_predictions_proba(self, texts, meta_embeddings, batch_size=16):
        """Get prediction probabilities with meta embeddings"""
        return safe_get_predictions_proba(self.model, self.tokenizer, texts, meta_embeddings,
                                        self.device, batch_size)

    def get_meta_embeddings_from_df(self, df):
        """Extract meta embeddings from DataFrame"""
        return precompute_meta_embeddings(df, self.meta_embedder, self.meta_cols)

    def optimize_thresholds_per_class(self, y_true, y_probs):
        """Optimize thresholds for each class using F1 maximization"""
        n_classes = len(self.label_cols)
        optimal_thresholds = np.zeros(n_classes)

        for class_idx in range(n_classes):
            precision, recall, thresholds = precision_recall_curve(
                y_true[:, class_idx], y_probs[:, class_idx]
            )

            # Calculate F1 scores
            f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)

            # Find threshold with maximum F1 score
            if len(thresholds) > 0:
                best_idx = np.nanargmax(f1_scores[:len(thresholds)])
                optimal_thresholds[class_idx] = thresholds[best_idx]

                print(f"{self.label_cols[class_idx]}: Optimal threshold = {thresholds[best_idx]:.3f}, "
                      f"Max F1 = {f1_scores[best_idx]:.3f}")
            else:
                optimal_thresholds[class_idx] = 0.5
                print(f"{self.label_cols[class_idx]}: No thresholds found, using default 0.5")

        return optimal_thresholds

    def evaluate_thresholds(self, y_true, y_probs, thresholds):
        """Evaluate performance with given thresholds"""
        y_pred = (y_probs >= thresholds).astype(int)

        metrics = {
            'f1_weighted': f1_score(y_true, y_pred, average='weighted'),
            'f1_macro': f1_score(y_true, y_pred, average='macro'),
            'f1_micro': f1_score(y_true, y_pred, average='micro'),
            'precision_weighted': precision_score(y_true, y_pred, average='weighted'),
            'recall_weighted': recall_score(y_true, y_pred, average='weighted'),
        }

        # Class-wise metrics
        class_metrics = {}
        for i, label in enumerate(self.label_cols):
            class_metrics[f'{label}_f1'] = f1_score(y_true[:, i], y_pred[:, i], zero_division=0)
            class_metrics[f'{label}_precision'] = precision_score(y_true[:, i], y_pred[:, i], zero_division=0)
            class_metrics[f'{label}_recall'] = recall_score(y_true[:, i], y_pred[:, i], zero_division=0)

        metrics.update(class_metrics)
        return metrics, y_pred

    def tune_thresholds_from_df(self, df, text_column='clean_text'):
        """Tune thresholds using DataFrame"""
        print("Extracting texts, meta embeddings, and true labels...")
        texts = df[text_column].tolist()
        meta_embeddings = self.get_meta_embeddings_from_df(df)
        y_true = df[self.label_cols].values

        print("Getting prediction probabilities...")
        y_probs = self.get_predictions_proba(texts, meta_embeddings)

        print(f"True labels shape: {y_true.shape}")
        print(f"Predicted probabilities shape: {y_probs.shape}")

        print("\nOptimizing thresholds...")
        self.best_thresholds = self.optimize_thresholds_per_class(y_true, y_probs)

        # Evaluate with optimized thresholds
        self.best_metrics, y_pred = self.evaluate_thresholds(y_true, y_probs, self.best_thresholds)

        # Compare with default threshold
        default_metrics, _ = self.evaluate_thresholds(y_true, y_probs, 0.5)

        print("\n" + "="*60)
        print("THRESHOLD TUNING RESULTS")
        print("="*60)

        print("\nOptimal thresholds:")
        for label, threshold in zip(self.label_cols, self.best_thresholds):
            print(f"  {label}: {threshold:.3f}")

        print("\nPerformance comparison:")
        print(f"{'Metric':<20} {'Default (0.5)':<12} {'Optimized':<12} {'Improvement':<12}")
        print("-" * 60)
        for metric in ['f1_weighted', 'f1_macro', 'f1_micro']:
            improvement = self.best_metrics[metric] - default_metrics[metric]
            print(f"{metric:<20} {default_metrics[metric]:<12.4f} {self.best_metrics[metric]:<12.4f} {improvement:+.4f}")

        print("\nClass-wise F1 scores:")
        for label in self.label_cols:
            default_f1 = default_metrics[f'{label}_f1']
            optimized_f1 = self.best_metrics[f'{label}_f1']
            improvement = optimized_f1 - default_f1
            print(f"  {label:<15} Default: {default_f1:.3f}, Optimized: {optimized_f1:.3f}, Δ: {improvement:+.3f}")

        return self.best_thresholds, self.best_metrics, y_pred

In [91]:
# Get the pre-computed meta embeddings for validation set
val_meta_emb = precompute_meta_embeddings(val_df_filtered, meta_embedder, meta_cols)

# Now call the function with meta embeddings
texts_val = val_df['clean_text'].tolist()
y_true_val = val_df[label_cols].values
y_probs_val = safe_get_predictions_proba(model, tokenizer, texts_val, val_meta_emb, batch_size=16, device='cuda')
print("Validation predictions shape:", y_probs_val.shape)

Getting predictions: 100%|██████████| 117/117 [00:15<00:00,  7.58it/s]

Validation predictions shape: (1867, 3)


In [94]:
tuner = MultilabelThresholdTuner(
    model=model,
    tokenizer=tokenizer,
    label_cols=label_cols,
    meta_embedder=meta_embedder,
    meta_cols=meta_cols,
    device='cuda'
)

best_thresholds, best_metrics, y_pred_val = tuner.tune_thresholds_from_df(val_df)
default_metrics, _ = tuner.evaluate_thresholds(y_true_val, y_probs_val, 0.5)

print("\nF1 Weighted Improvement over default 0.5 threshold:",
      best_metrics['f1_weighted'] - default_metrics['f1_weighted'])

Extracting texts, meta embeddings, and true labels...
Getting prediction probabilities...


Getting predictions: 100%|██████████| 117/117 [00:18<00:00,  6.49it/s]


True labels shape: (1867, 3)
Predicted probabilities shape: (1867, 3)

Optimizing thresholds...
is_ad: Optimal threshold = 0.756, Max F1 = 0.083
is_relevant: Optimal threshold = 0.000, Max F1 = 0.981
is_rant: Optimal threshold = 0.852, Max F1 = 0.186

THRESHOLD TUNING RESULTS

Optimal thresholds:
  is_ad: 0.756
  is_relevant: 0.000
  is_rant: 0.852

Performance comparison:
Metric               Default (0.5) Optimized    Improvement 
------------------------------------------------------------
f1_weighted          0.7206       0.8983       +0.1777
f1_macro             0.3332       0.4168       +0.0836
f1_micro             0.4469       0.7163       +0.2694

Class-wise F1 scores:
  is_ad           Default: 0.068, Optimized: 0.083, Δ: +0.015
  is_relevant     Default: 0.788, Optimized: 0.981, Δ: +0.194
  is_rant         Default: 0.144, Optimized: 0.186, Δ: +0.042

F1 Weighted Improvement over default 0.5 threshold: 0.17770235344796392


In [95]:
for label in label_cols:
    default_f1 = default_metrics[f'{label}_f1']
    optimized_f1 = best_metrics[f'{label}_f1']
    print(f"{label}: Default F1 = {default_f1:.3f}, Optimized F1 = {optimized_f1:.3f}")

is_ad: Default F1 = 0.068, Optimized F1 = 0.083
is_relevant: Default F1 = 0.788, Optimized F1 = 0.981
is_rant: Default F1 = 0.144, Optimized F1 = 0.186


In [96]:
y_pred_final = (y_probs_val >= tuner.best_thresholds).astype(int)

In [97]:
thresholds_to_save = dict(zip(label_cols, tuner.best_thresholds))
with open("multilabel_thresholds.json", "w") as f:
    json.dump(thresholds_to_save, f)
files.download("multilabel_thresholds.json")

print("Optimized thresholds saved to multilabel_thresholds.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Optimized thresholds saved to multilabel_thresholds.json


In [98]:
# Monitor class distribution in production
class_distribution = {
    'is_ad': (y_pred_val[:, 0].sum() / len(y_pred_val)),
    'is_relevant': (y_pred_val[:, 1].sum() / len(y_pred_val)),
    'is_rant': (y_pred_val[:, 2].sum() / len(y_pred_val))
}
print("Predicted class distribution:", class_distribution)

Predicted class distribution: {'is_ad': np.float64(0.35511515800749865), 'is_relevant': np.float64(1.0), 'is_rant': np.float64(0.44349223352972683)}


### Testing Model

In [101]:
def predict_is_legit(ad_preds, relevant_preds, rant_preds):
    """
    Predict is_legit based on business rules:
    - If is_ad = True → is_legit = False
    - If is_relevant = False → is_legit = False
    - If is_rant = True → is_legit = False
    - Otherwise → is_legit = True
    """
    is_legit = np.ones_like(ad_preds, dtype=bool)  # Start with all True

    # Apply business rules
    is_legit[ad_preds == 1] = False      # If ad → not legit
    is_legit[relevant_preds == 0] = False # If not relevant → not legit
    is_legit[rant_preds == 1] = False    # If rant → not legit

    return is_legit.astype(int)

# Or using vectorized operations:
def predict_is_legit_vectorized(ad_preds, relevant_preds, rant_preds):
    """Vectorized version for better performance"""
    return ((ad_preds == 0) & (relevant_preds == 1) & (rant_preds == 0)).astype(int)

def predict_with_business_rules(model, tokenizer, texts, meta_embeddings, thresholds, device='cuda', batch_size=16):
    """
    Complete prediction pipeline with business rules for is_legit
    """
    # Get probability predictions
    probs = safe_get_predictions_proba(model, tokenizer, texts, meta_embeddings, device, batch_size)

    # Apply individual class thresholds
    ad_preds = (probs[:, 0] >= thresholds['is_ad']).astype(int)
    relevant_preds = (probs[:, 1] >= thresholds['is_relevant']).astype(int)
    rant_preds = (probs[:, 2] >= thresholds['is_rant']).astype(int)

    # Apply business rules for is_legit
    is_legit_preds = predict_is_legit(ad_preds, relevant_preds, rant_preds)

    return {
        'is_ad': ad_preds,
        'is_relevant': relevant_preds,
        'is_rant': rant_preds,
        'is_legit': is_legit_preds,
        'probabilities': probs
    }


In [108]:
def robust_predict_with_business_rules(model, tokenizer, texts, meta_embeddings, thresholds, device='cuda', batch_size=16):
    """
    Robust prediction function that handles both array and dictionary thresholds
    """
    # Get probability predictions
    probs = safe_get_predictions_proba(model, tokenizer, texts, meta_embeddings, device, batch_size)

    # Handle threshold format
    if isinstance(thresholds, np.ndarray):
        # Convert array to dictionary
        threshold_dict = {
            'is_ad': float(thresholds[0]),
            'is_relevant': float(max(thresholds[1], 0.01)),  # Apply minimum constraint
            'is_rant': float(thresholds[2])
        }
    elif isinstance(thresholds, dict):
        # Use dictionary directly, but apply constraint to is_relevant
        threshold_dict = thresholds.copy()
        threshold_dict['is_relevant'] = max(threshold_dict.get('is_relevant', 0.5), 0.1)
    else:
        raise ValueError("Thresholds must be numpy array or dictionary")

    print(f"Using thresholds: {threshold_dict}")

    # Apply individual class thresholds
    ad_preds = (probs[:, 0] >= threshold_dict['is_ad']).astype(int)
    relevant_preds = (probs[:, 1] >= threshold_dict['is_relevant']).astype(int)
    rant_preds = (probs[:, 2] >= threshold_dict['is_rant']).astype(int)

    # Apply business rules for is_legit
    is_legit_preds = ((ad_preds == 0) & (relevant_preds == 1) & (rant_preds == 0)).astype(int)

    return {
        'is_ad': ad_preds,
        'is_relevant': relevant_preds,
        'is_rant': rant_preds,
        'is_legit': is_legit_preds,
        'probabilities': probs,
        'thresholds_used': threshold_dict
    }

# Test with the array directly
val_predictions = robust_predict_with_business_rules(
    model=model,
    tokenizer=tokenizer,
    texts=val_df['clean_text'].tolist(),
    meta_embeddings=val_meta_emb,
    thresholds=best_thresholds
)

Getting predictions: 100%|██████████| 117/117 [00:49<00:00,  2.36it/s]

Using thresholds: {'is_ad': 0.756177544593811, 'is_relevant': 0.1, 'is_rant': 0.8518295884132385}


In [109]:
# Analyze the predictions
print("Prediction results:")
for label in ['is_ad', 'is_relevant', 'is_rant', 'is_legit']:
    preds = val_predictions[label]
    percentage = preds.mean() * 100
    print(f"{label}: {preds.sum()}/{len(preds)} ({percentage:.1f}%)")

# Check threshold values used
print("\nThresholds used:")
for label, threshold in val_predictions['thresholds_used'].items():
    print(f"  {label}: {threshold:.3f}")

Prediction results:
is_ad: 656/1867 (35.1%)
is_relevant: 1546/1867 (82.8%)
is_rant: 826/1867 (44.2%)
is_legit: 764/1867 (40.9%)

Thresholds used:
  is_ad: 0.756
  is_relevant: 0.100
  is_rant: 0.852
